In [4]:
from dotenv import load_dotenv,find_dotenv
from langchain_openai.embeddings import OpenAIEmbeddings
import pandas as pd
import numpy as np
import openai
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.vectorstores import FAISS

In [2]:
load_dotenv(dotenv_path=find_dotenv(filename="../.env"))

True

In [4]:
embeddings=OpenAIEmbeddings()

In [8]:
df=pd.read_excel(io="data/Data.xlsx")

In [9]:
df

,Words
0,Elephant
1,Lion
2,Tiger
3,Dog
4,Cricket
5,Footbal
6,Tennis
7,Basketball
8,Apple
9,Orange


In [13]:
df['embeddings']=df['Words'].apply(lambda x: embeddings.embed_query(text=x))

In [14]:
df

,Words,embeddings
0,Elephant,"[-0.018824968530431193, -0.008682483827226351,..."
1,Lion,"[-0.0015009929272673714, -0.010024921244147899..."
2,Tiger,"[-0.013500550598151365, -0.009651595344086194,..."
3,Dog,"[-0.0008935772431031664, -0.015069474605842001..."
4,Cricket,"[0.003957032484809424, -0.007208025088677429, ..."
5,Footbal,"[-0.011442113927512236, -0.008124158902145616,..."
6,Tennis,"[-0.023049057156337614, 0.0015891833895531008,..."
7,Basketball,"[-0.012911115406280736, -0.013261756483018321,..."
8,Apple,"[0.014532958590987296, -0.003988702690459901, ..."
9,Orange,"[0.020821801954896618, -0.029376701870735933, ..."


In [12]:
ourQuery="Mango"

In [16]:
ourQueryEmbedding=embeddings.embed_query(text=ourQuery)

In [17]:
print(ourQueryEmbedding)

[-0.0034238057053754753, -0.019852961713654013, 0.010443884547284227, -0.0161608731823682, 0.006323820244618822, 0.008610615232991139, -0.02519307764238657, -0.014244564355994938, 0.0010843115979634825, -0.0330116179520127, -0.001220848605335037, 0.0037591598198399873, 0.0019258906219622163, 0.016837968886972133, -0.021615966467216745, 0.004813129860609791, 0.03265390813834727, -0.008131538259228459, 0.009492117609772505, -0.015713734052641336, -0.01596924239286313, 0.0027866327714775848, 0.001406091802749882, -0.01248156053375475, 0.004618305100436628, 0.009555995160489225, 0.024541531840482272, -0.01363134527078518, 0.0116128331083232, 0.0030229777533521064, 0.04795883070927884, -0.015023863396687587, -0.00429892014082066, -0.009907317964141009, -0.017553392239593152, 0.004637467993122627, -0.010354457093867873, -0.006358952524984, -0.006256749188895283, -0.01834546530031307, 0.01441064431147783, -0.00834871988275571, -0.015700960032614064, -0.018013305389347287, -0.01545822664374209

In [21]:
cosineList=[]
magnitudeQueryEmbedding=sum([b*b for b in ourQueryEmbedding])**0.5
for matrix in df['embeddings'].values:
    dot=sum([a*b for a,b in zip(matrix,ourQueryEmbedding)])
    
    magnitudeMatrix=sum([a*a for a in matrix])**0.5
    cosine=dot/(magnitudeMatrix*magnitudeQueryEmbedding)
    cosineList.append(cosine)

In [22]:
df['cosineValues']=cosineList

In [23]:
df

,Words,embeddings,cosineValues
0,Elephant,"[-0.018824968530431193, -0.008682483827226351,...",0.831075
1,Lion,"[-0.0015009929272673714, -0.010024921244147899...",0.826777
2,Tiger,"[-0.013500550598151365, -0.009651595344086194,...",0.851483
3,Dog,"[-0.0008935772431031664, -0.015069474605842001...",0.771768
4,Cricket,"[0.003957032484809424, -0.007208025088677429, ...",0.817301
5,Footbal,"[-0.011442113927512236, -0.008124158902145616,...",0.776767
6,Tennis,"[-0.023049057156337614, 0.0015891833895531008,...",0.804800
7,Basketball,"[-0.012911115406280736, -0.013261756483018321,...",0.793779
8,Apple,"[0.014532958590987296, -0.003988702690459901, ...",0.813071
9,Orange,"[0.020821801954896618, -0.029376701870735933, ...",0.843466


In [25]:
df.sort_values(by="cosineValues",ascending=False)

,Words,embeddings,cosineValues
10,Banana,"[-0.013107325443239795, -0.020157383080523828,...",0.898264
2,Tiger,"[-0.013500550598151365, -0.009651595344086194,...",0.851483
9,Orange,"[0.020821801954896618, -0.029376701870735933, ...",0.843466
0,Elephant,"[-0.018824968530431193, -0.008682483827226351,...",0.831075
1,Lion,"[-0.0015009929272673714, -0.010024921244147899...",0.826777
4,Cricket,"[0.003957032484809424, -0.007208025088677429, ...",0.817301
8,Apple,"[0.014532958590987296, -0.003988702690459901, ...",0.813071
6,Tennis,"[-0.023049057156337614, 0.0015891833895531008,...",0.804800
7,Basketball,"[-0.012911115406280736, -0.013261756483018321,...",0.793779
5,Footbal,"[-0.011442113927512236, -0.008124158902145616,...",0.776767


In [26]:
df.drop(columns=['cosineValues']).to_csv(
    path_or_buf="output/wordembeddings.csv",
    index=False)

In [2]:
loader=CSVLoader(
    file_path="./SimilarWordsFinderApp/mydata.csv",
    csv_args={
        "delimiter":",",
        "quotechar": '"',
        "fieldnames":['Words']
    }
)

In [3]:
data=loader.load()


In [6]:
embeddings=OpenAIEmbeddings()
db=FAISS.from_documents(
    documents=data,
    embedding=embeddings,
)

In [7]:
data

[Document(page_content='Words: Words', metadata={'source': './SimilarWordsFinderApp/mydata.csv', 'row': 0}),
 Document(page_content='Words: Elephant', metadata={'source': './SimilarWordsFinderApp/mydata.csv', 'row': 1}),
 Document(page_content='Words: Lion', metadata={'source': './SimilarWordsFinderApp/mydata.csv', 'row': 2}),
 Document(page_content='Words: Tiger', metadata={'source': './SimilarWordsFinderApp/mydata.csv', 'row': 3}),
 Document(page_content='Words: Dog', metadata={'source': './SimilarWordsFinderApp/mydata.csv', 'row': 4}),
 Document(page_content='Words: Cricket', metadata={'source': './SimilarWordsFinderApp/mydata.csv', 'row': 5}),
 Document(page_content='Words: Footbal', metadata={'source': './SimilarWordsFinderApp/mydata.csv', 'row': 6}),
 Document(page_content='Words: Tennis', metadata={'source': './SimilarWordsFinderApp/mydata.csv', 'row': 7}),
 Document(page_content='Words: Basketball', metadata={'source': './SimilarWordsFinderApp/mydata.csv', 'row': 8}),
 Document

In [8]:
db.similarity_search(query="Mango")

[Document(page_content='Words: Banana', metadata={'source': './SimilarWordsFinderApp/mydata.csv', 'row': 11}),
 Document(page_content='Words: Orange', metadata={'source': './SimilarWordsFinderApp/mydata.csv', 'row': 10}),
 Document(page_content='Words: Apple', metadata={'source': './SimilarWordsFinderApp/mydata.csv', 'row': 9}),
 Document(page_content='Words: Tiger', metadata={'source': './SimilarWordsFinderApp/mydata.csv', 'row': 3})]